# install libs

In [1]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.2 MB/s eta 0:00:00


In [2]:
!pip install transformers torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.5 MB/s eta 0:00:00


In [3]:
!python -m spacy download fr_core_news_sm

2023-04-16 19:41:51.278734: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 19:41:53.057904: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-16 19:41:55.781098: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-16 19:41:55.781759: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [4]:
import glob
import json
import spacy

nlp = spacy.load("fr_core_news_sm")


In [5]:
import pandas as pd
from tqdm.auto import tqdm

# load pipeline

In [6]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis", model="cmarkea/distilcamembert-base-sentiment", device=0)

# define enrichment process

In [9]:
# still not tested code
from spacy import displacy

def enrich_with_relevant_metrics(input_file_path, output_file_path):
    with open(input_file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    text = data["question"]["textesReponse"]["texteReponse"]["texte"]
    doc = nlp(text)

    # Sentiment and confidence score
    sentiment_data = []
    for sent in doc.sents:
        sentiment = sentiment_pipeline(sent.text)[0]
        label = sentiment["label"]
        score = sentiment["score"]

        sentiment_data.append({
            "begin_char": sent.start_char,
            "end_char": sent.end_char,
            "sentiment": label,
            "score": score,
        })

    data["question"]["textesReponse"]["texteReponse"]["sentiment_data"] = sentiment_data

    with open(output_file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


# perform enrichment

In [10]:
from tqdm.auto import tqdm
import os
import glob

input_folder = "/content/drive/MyDrive/AI/QAG/enrichment/qag_json/qag_json/"
output_folder = "/content/drive/MyDrive/AI/QAG/enrichment/processed_5"

os.makedirs(output_folder, exist_ok=True)

input_files = glob.glob(os.path.join(input_folder, "*.json"))

for input_file in tqdm(input_files):
    file_name = os.path.basename(input_file)
    output_file = os.path.join(output_folder, file_name)
    enrich_with_relevant_metrics(input_file, output_file)


  0%|          | 0/639 [00:00<?, ?it/s]